In [1]:
import osmnx as ox
import networkx as nx
import igraph as ig
import pandas as pd
import numpy as np

In [ ]:
def change_type(x):
    try:
        y = int(x)
        return y
    except:
        # print(x)
        return -1
def get_seconds(x):
    return pd.Timedelta(x).total_seconds()

In [ ]:
data = pd.read_csv('./share_data/day_1_15_5_0', names=['D', 'O', 'no_use_time', 'share_ot', 'share_dt'])
# 连接数
r1 = len(data)
data['new_D'] = data['D'].apply(change_type)
data['new_O'] = data['O'].apply(change_type)
new_data = data[~((data['new_D'] == -1) | (data['new_O'] == -1))]
new_data = new_data.drop_duplicates(subset=['new_D', 'new_O'])
# 有效连接数
r2 = len(new_data)
# 节点数
r3 = len(set(new_data['new_D']) | set(new_data['new_O']))
new_data['no_use_time'] = new_data['no_use_time'].apply(get_seconds).astype('int32')
G = nx.DiGraph()
G.add_weighted_edges_from([(O, D, weight) for O, D, weight in zip(new_data['new_O'], new_data['new_D'], new_data['no_use_time'])])
# 拆出来的新点用原始点的字符串
new_edges = []
left, right = [], []
for edge in G.edges():
    edge_end = str(edge[1])
    new_edge = (edge[0], edge_end)
    new_edges.append(new_edge)
    left.append(edge[0])
    right.append(edge_end)
# 构建二分图（有向的）
new_G = nx.DiGraph()
new_G.add_nodes_from(left, bipartite=0)
new_G.add_nodes_from(right, bipartite=1)
new_G.add_weighted_edges_from([(O, D, weight) for (O, D), weight in zip(new_edges, new_data['no_use_time'])])
nx.is_bipartite(new_G)
# 注意此处的匹配是重复了的
match = nx.bipartite.hopcroft_karp_matching(new_G, left)
match_dict = {k:v for k,v in match.items() if isinstance(k, int)}
# 匹配数
r4 = len(match_dict)
# 最小覆盖路径数
r6 = G.number_of_nodes() - len(match_dict)
# 基于匹配的边获取调度行程线路
path = [[k, int(v)] for k,v in match_dict.items()]
while True:
    o = [r[0] for r in path]
    d = [r[-1] for r in path]
    com = set(d) & set(o)
    new_route = []
    left_route = []
    right_route = []
    if com:
        for c in com:
            lf = path[d.index(c)]
            rt = path[o.index(c)]
            if lf not in (left_route + right_route) and rt not in (left_route + right_route):
                left_route.append(lf)
                right_route.append(rt)
                new_route.append(lf[:-1] + rt)
        for i in range(len(new_route)):
            path.remove(left_route[i])
            path.remove(right_route[i])
            path.append(new_route[i])
    else:
        break
s = 0
match_nodes = []
for r in path:
    s += (len(r) -1)
    match_nodes.extend(r)
# 匹配路径数
r7 = len(path)
# 匹配节点数
r5 = len(match_nodes)
new_data.set_index([new_data.new_O, new_data.new_D], drop=True, inplace=True)
ind = [(k, int(v)) for k,v in match_dict.items()]
no_use_time_sum = new_data.loc[ind, 'no_use_time'].sum()
# 匹配节点的空驶时间
r8 = no_use_time_sum
# 匹配节点的空驶时间
r8 = no_use_time_sum
r9 = r8/r5
pd.DataFrame([r1, r2, r3, r4, r5, r6, r7, r8, r9]).to_csv(r'C:\Users\Administrator\Desktop\canshu.csv', header=None, index=False, mode='a')